# Who is heading for Diabetes?

This is the predictive part of the 2017 Melbourne Datathon.

The task is to predict the probability that a patient will be dispensed a drug related to Diabetes post 2015. This is quite important research as it will be an early warning system for doctors so intervention can potentially be made before it is too late.

Use the patients that we have provided all the records for to build your model, then see how it performs on these unseen people.

For patient ID'S 279,201 to 558,352 you need to submit a file with 2 columns, the Patient_ID and the probability in the range [0-1]. The file will have 279,153 rows including the header row. An example submission file is provided for download.

In [ ]:
import tqdm

import pandas as pd
import numpy as np
import pickle
import sqlite3
import xgboost

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import normalize, StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

pd.options.display.max_columns = 50
np.set_printoptions(suppress=True)

## Data retrieval functions

In [ ]:
conn = sqlite3.connect("../data/datasets.db")

In [ ]:
def retrieve_patient_data(connection, patient_id):
    """
    Return the patient data.
    """
    SQL = """
SELECT *
FROM transactions a
LEFT OUTER JOIN ChronicIllness_LookUp b 
    ON a.Drug_ID = b.MasterProductID 
LEFT OUTER JOIN patients c
    ON a.Patient_ID = c.Patient_ID
LEFT OUTER JOIN classification d
    ON a.Patient_ID = d.Patient_ID
LEFT OUTER JOIN social e
    ON c.postcode = e.postcode
WHERE a.Patient_ID = {}
AND a.prescription_week < '2016-01-01'
ORDER BY prescription_week
    """.format(patient_id)

    return pd.read_sql_query(SQL, connection)


def retrieve_chronic_illness(connection):
    SQL = """
SELECT DISTINCT ChronicIllness FROM ChronicIllness_LookUp 
    """.format(patient_id)
    return pd.read_sql_query(SQL, connection)
    
    

## Feature extraction functions

In [ ]:
gender_map = {'F': 1, 'M': 0, 'U': 0.5}

def gender(patient_frame):
    return gender_map[patient_frame.gender[0]]

def age(patient_frame):
    patient_age = 2016 - patient_frame.year_of_birth[0]
    if patient_age > 100: 
        return 0.5 
    else: 
        return patient_age / 100.

def age_group(patient_frame):
    patient_age = 2016 - patient_frame.year_of_birth[0]
    if patient_age in range(0, 18):
        return 1
    elif patient_age in range(18, 35):
        return 2
    elif patient_age in range(35, 55):
        return 3
    elif patient_age in range(55, 70):
        return 4
    elif patient_age in range(70, 100):
        return 5
    else:
        return 0
    
def socio_scores(patient_frame): 
    scores = {
        'disadvantage_score': 0.,
        'advantage_score': 0.,
        'economic_score': 0.,
        'occupation_score': 0.,
        'population': 0.
    }
    
    def get_score(element):        
        score = patient_frame[element][0]
        if isinstance(score, str):
            return 1.
        if score is None:
            return 1.
        return float(score) / 1000.
    
    scores.update({key: get_score(key) for key in scores})
    return scores

def diabetes(patient_frame):
    return float(patient_frame.ChronicIllness.str.contains('Diabetes').any())

def mean_script_time(patient_frame):
    script_week_diff = pd.to_datetime(patient_frame.Prescription_Week).diff()
    return script_week_diff[script_week_diff > pd.Timedelta(0)].mean() / pd.Timedelta(days=365*6)

def system_codes(patient_frame):
    codes = {
         'A': 0, # Unknown
         'C': 0, # Unknown   
         'Z': 0, # Unknown
         'N': 0, # NHS Script
         'P': 0, # Private Script
         'B': 0, # Doctors Bag Script
         'T': 0, # Schedule Three Recordable Script
         'R': 0, # Repatriation Script
         'D': 0, # Dental Script
         'E': 0, # Optometrist Script
         'U': 0, # Nurse Practitioner Script
         'F': 0  # Midwife Script
        }

    codes.update(patient_frame.SourceSystem_Code.value_counts(normalize=True).to_dict())
    return codes

def chronic_illness(patient_frame):
    chronic_treatments = {
     'Anti-Coagulant': 0,
     'Chronic Obstructive Pulmonary Disease (COPD)': 0,
     'ChronicIllness': 0,
     'Depression': 0,
     'Diabetes': 0,
     'Epilepsy': 0,
     'Heart Failure': 0,
     'Hypertension': 0,
     'Immunology': 0,
     'Lipids': 0,
     'Osteoporosis': 0,
     'Urology': 0}
    chronic_treatments.update(patient_frame.ChronicIllness.value_counts(normalize=True).to_dict())
    return chronic_treatments

## Compute some basic features of the data 

In [ ]:
def feature_extract(patient_frame):
    """
    Form a feature dictionary
    """
    
    feature_dict = {
        'gender': gender(patient_frame), 
        'age': age(patient_frame), 
        'age_group': age_group(patient_frame),
        'mean_script_time': mean_script_time(patient_frame),
        'target': patient_frame.Target[0]}
    
    feature_dict.update(system_codes(patient_frame))
    feature_dict.update(chronic_illness(patient_frame)) 
    feature_dict.update(socio_scores(patient_frame))
    
    return feature_dict
    

## Extract features of the data

Perform a random sample of patients.

In [ ]:
#n = 100000
#patient_ids = np.random.randint(0, 279201, n)
#patient_data = []
#for patient_id in tqdm.tqdm_notebook(patient_ids):
#    patient_data.append(retrieve_patient_data(conn, patient_id))

In [ ]:
#with open('../submissions/patient_data.pkl', 'wb') as fh:
#    pickle.dump(patient_data, fh)

In [ ]:
#with open('../submissions/patient_data.pkl', 'rb') as fh:
#    patient_data = pickle.load(fh)

In [ ]:
def extract_features(n_patients):
    """Samples patients and extracts features in one step to save memory"""
    patient_ids = np.random.randint(0, 279201, n_patients)
    features = []
    for patient_id in tqdm.tqdm_notebook(patient_ids):
        patient_data = retrieve_patient_data(conn, patient_id)
        if len(patient_data):
            features.append(feature_extract(patient_data))
        
    return pd.DataFrame(features)

In [ ]:
n = 200000

feature_frame = extract_features(n)

Extract features into a feature dataframe.

In [ ]:
#features = []
#for patient in tqdm.tqdm_notebook(patient_data): 
#    if len(patient):
#        features.append(feature_extract(patient))
#feature_frame = pd.DataFrame(features)

Store the features and ?patient data? to disk.

In [ ]:
feature_frame.to_csv('../submissions/features.csv')

In [ ]:
#feature_frame = pd.read_csv('../submissions/features.csv')

## Explore the feature we have extracted

Note: not sure here - trying out something from scikit learn but it may not be sensible.

In [ ]:
feature_frame.dropna(inplace=True)
features = feature_frame.iloc[:, 0:len(feature_frame.columns) - 1]
target = feature_frame['target']

Ranking based on ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

etc = ExtraTreesClassifier()
etc.fit(features.values, target.values)
print(pd.Series(etc.feature_importances_, index=features.columns).sort_values())

In [ ]:
etc_top_features = pd.Series(etc.feature_importances_, index=features.columns).sort_values(ascending=False)[0:10].index

Ranking based on RFE

In [ ]:
# Feature Extraction with RFE
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# feature extraction
model = LogisticRegression()
rfe = RFE(model, 10)
rfe_fit = rfe.fit(features.values, target.values)
print("Num Features: {}".format(rfe_fit.n_features_))
print("Selected Features: \n{}".format(pd.Series(rfe_fit.support_, index=features.columns)))
print("Feature Ranking: {}".format(rfe_fit.ranking_))

Ranking based on SelectKBest

This doesn't seem to deal with with some of the features but I haven't taken the time to investigate

In [ ]:
# Feature Extraction with Univariate Statistical Tests (Chi-squared for classification)
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# feature extraction
kbest = SelectKBest(score_func=chi2, k=10)
kbest_fit = kbest.fit(features, target)

# summarize scores
print(pd.Series(kbest_fit.scores_, index=features.columns).sort_values())
reduced_features = kbest_fit.transform(features.values)
# summarize selected features
print(reduced_features[0:9,:])

Using PCA to extract features

In [ ]:
# Feature Extraction with PCA
from sklearn.decomposition import PCA

# feature extraction
pca = PCA(n_components=2)
pca_fit = pca.fit(features.values)
# summarize components
print("Explained Variance: {}".format(pca_fit.explained_variance_ratio_))
print(pca_fit.components_)

Nathan's feature summary

In [ ]:
means = feature_frame.groupby('target').mean()
means

Choose a method to cull features

In [ ]:
# select important features based on ETC
#features = feature_frame[etc_top_features]

In [ ]:
# drop unimportant features based on RFE
rfe_support = pd.Series(rfe_fit.support_, index=features.columns)
features = feature_frame[rfe_support[rfe_support == True].index]

## Try out a set of different classifiers

Note: the feature matrix is usually transformed to have zero mean and unit standard deviation.

In [ ]:
X = features.dropna().values #feature_frame.dropna().drop('target', axis=1).values
y = feature_frame.dropna().target.values

In [ ]:
# Uncomment this to use features based on PCA
#pca = PCA(n_components=4)
#pca_fit = pca.fit(X)
#X_transformed = pca_fit.transform(X, y)

# Uncomment this to use features based on SelectKBest
#kbest = SelectKBest(score_func=chi2, k=10)
#kbest_fit = kbest.fit(X, y)
#X_transformed = kbest_fit.transform(X)

X_transformed = StandardScaler().fit_transform(X)

Partition the data into test and train datasets.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.5, random_state=0)

In [ ]:
classifiers = [
    ('Random Forrest', RandomForestClassifier()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Adaboost',AdaBoostClassifier() ),
    ('xgboost Gradient boosted decition tree', xgboost.XGBClassifier()), 
    ('Gradient boosting classifier', GradientBoostingClassifier()),    
    ('Bagging', BaggingClassifier()),
    ('Bernoulli Naive Bayes', BernoulliNB()),
    ('MLPClassifier (NN)', MLPClassifier()),
    ('LDA', LinearDiscriminantAnalysis())
    #('SVM',SVC(probability=True) # too slow!
]

for name, clf in classifiers:
    print('Classifier: {}'.format(name))
    model = clf.fit(X_train, y_train)
    y_true, y_pred = y_test, clf.predict(X_test) 
    score = model.score(X_test, y_test)
    print('Score {}'.format(score))
    print(classification_report(y_true, y_pred))


## Form a submission

Perform the prediction in 1000 patient "chunks" to speed up the processing.

In [ ]:
model = RandomForestClassifier().fit(X, y)#xgboost.XGBClassifier().fit(X, y)

In [ ]:
submission = pd.read_csv('../submissions/diabetes_submission_example.csv')

In [ ]:
chunks = submission.groupby(np.arange(len(submission)) // 1000)

In [ ]:
for group, frame in tqdm.tqdm_notebook(chunks):
    
    # Extract the features
    data = [feature_extract(retrieve_patient_data(conn, x)) for x in frame.Patient_ID.values]
    
    # Construct prediction X matrix
    features = pd.DataFrame(data)
    
    # filter features
    features = features[rfe_support[rfe_support == True].index]    
    
    pred_x = features.values #features.drop('target', axis=1).values
    pred_x[np.isnan(pred_x)] = 0.0
    
    # Apply the standard transform prior to fitting. 
    pred_x = StandardScaler().fit_transform(pred_x)
    
    # Fit the model
    submission.Diabetes[frame.index] = model.predict_proba(pred_x)[:, 0]

In [ ]:
submission.to_csv('../submissions/kaggle_four.csv', index=False)